# 🚀 PPuRI-AI Ultimate 서버 제어판

이 노트북을 통해 PPuRI-AI Ultimate 서버를 쉽게 시작, 중지, 모니터링할 수 있습니다.

## 📋 기능
- 🟢 서버 시작/중지
- 📊 실시간 상태 모니터링  
- 🔍 로그 확인
- 🌐 브라우저 바로가기
- 🧪 API 테스트

In [ ]:
# 필요한 라이브러리 설치 및 임포트
import subprocess
import time
import os
import signal
import requests
import json
from pathlib import Path
from IPython.display import display, HTML, clear_output
import threading

# 프로젝트 디렉토리 설정
PROJECT_DIR = Path.cwd()
PID_FILE = PROJECT_DIR / "server_notebook.pid"
SERVER_URL = "http://localhost:8002"

print("✅ 라이브러리 로드 완료")
print(f"📁 프로젝트 디렉토리: {PROJECT_DIR}")

## 🎛️ 서버 제어

In [ ]:
def start_server():
    """서버 시작"""
    if is_server_running():
        print("⚠️ 서버가 이미 실행 중입니다.")
        return
    
    try:
        # 백그라운드에서 서버 실행
        process = subprocess.Popen(
            ["python3", "simple_stable_server.py"],
            cwd=PROJECT_DIR,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            preexec_fn=os.setsid
        )
        
        # PID 저장
        with open(PID_FILE, 'w') as f:
            f.write(str(process.pid))
        
        print(f"🚀 서버 시작됨 (PID: {process.pid})")
        print("⏱️ 서버가 시작되는 동안 잠시 기다려주세요...")
        
        # 서버 시작 대기
        for i in range(10):
            time.sleep(1)
            if check_server_health():
                print("✅ 서버가 성공적으로 시작되었습니다!")
                display_server_links()
                return
        
        print("⚠️ 서버 시작 확인 중... 직접 브라우저에서 확인해주세요.")
        
    except Exception as e:
        print(f"❌ 서버 시작 실패: {e}")

def stop_server():
    """서버 중지"""
    if not PID_FILE.exists():
        print("⚠️ 실행 중인 서버가 없습니다.")
        return
    
    try:
        with open(PID_FILE, 'r') as f:
            pid = int(f.read().strip())
        
        # 프로세스 그룹 종료
        os.killpg(os.getpgid(pid), signal.SIGTERM)
        PID_FILE.unlink()
        
        print("🛑 서버가 중지되었습니다.")
        
    except Exception as e:
        print(f"❌ 서버 중지 실패: {e}")
        if PID_FILE.exists():
            PID_FILE.unlink()

def is_server_running():
    """서버 실행 상태 확인"""
    if not PID_FILE.exists():
        return False
    
    try:
        with open(PID_FILE, 'r') as f:
            pid = int(f.read().strip())
        os.kill(pid, 0)
        return True
    except:
        if PID_FILE.exists():
            PID_FILE.unlink()
        return False

def check_server_health():
    """서버 헬스 체크"""
    try:
        response = requests.get(f"{SERVER_URL}/api/health", timeout=2)
        return response.status_code == 200
    except:
        return False

def display_server_links():
    """서버 링크 표시"""
    html = f"""
    <div style="padding: 15px; border: 2px solid #28a745; border-radius: 10px; background-color: #d4edda;">
        <h3 style="color: #155724; margin-top: 0;">🌐 서버 접속 링크</h3>
        <p><strong>메인 페이지:</strong> <a href="{SERVER_URL}" target="_blank">{SERVER_URL}</a></p>
        <p><strong>헬스 체크:</strong> <a href="{SERVER_URL}/api/health" target="_blank">{SERVER_URL}/api/health</a></p>
        <p><strong>API 문서:</strong> <a href="{SERVER_URL}/docs" target="_blank">{SERVER_URL}/docs</a></p>
        <p><strong>상태 정보:</strong> <a href="{SERVER_URL}/api/status" target="_blank">{SERVER_URL}/api/status</a></p>
    </div>
    """
    display(HTML(html))

print("✅ 서버 제어 함수 준비 완료")

## 🚀 서버 시작

In [ ]:
# 서버 시작
start_server()

## 📊 서버 상태 확인

In [ ]:
def show_server_status():
    """서버 상태 표시"""
    if is_server_running():
        print("🟢 서버 실행 중")
        
        if check_server_health():
            print("✅ 서버 헬스 체크 통과")
            display_server_links()
            
            # 서버 상태 정보 가져오기
            try:
                response = requests.get(f"{SERVER_URL}/api/status")
                if response.status_code == 200:
                    data = response.json()
                    print("\n📋 서버 상태 정보:")
                    print(f"  - 상태: {data.get('server_status', 'unknown')}")
                    print(f"  - 포트: {data.get('port', 'unknown')}")
                    
                    features = data.get('features', {})
                    if 'agno_agents' in features:
                        agno = features['agno_agents']
                        print(f"  - Agno 에이전트: {agno.get('status', 'unknown')}")
                        print(f"  - 생성 시간: {agno.get('creation_time', 'unknown')}")
                        print(f"  - 메모리 사용량: {agno.get('memory_usage', 'unknown')}")
            except Exception as e:
                print(f"⚠️ 상태 정보 가져오기 실패: {e}")
        else:
            print("⚠️ 서버가 실행 중이지만 응답하지 않습니다.")
    else:
        print("🔴 서버가 실행되지 않았습니다.")

show_server_status()

## 🧪 API 테스트

In [ ]:
def test_api():
    """API 테스트"""
    if not check_server_health():
        print("❌ 서버가 실행되지 않았습니다.")
        return
    
    print("🧪 API 테스트 시작...")
    
    # 1. 헬스 체크
    try:
        response = requests.get(f"{SERVER_URL}/api/health")
        print(f"✅ 헬스 체크: {response.status_code}")
    except Exception as e:
        print(f"❌ 헬스 체크 실패: {e}")
    
    # 2. 로그인 테스트
    try:
        login_data = {"username": "admin_001", "password": "admin_pass_001"}
        response = requests.post(f"{SERVER_URL}/api/auth/login", json=login_data)
        print(f"✅ 로그인 테스트: {response.status_code}")
        
        if response.status_code == 200:
            token = response.json().get('data', {}).get('access_token')
            print(f"🔑 토큰 획득: {token[:20]}...")
            
            # 3. 에이전트 목록 조회
            headers = {"Authorization": f"Bearer {token}"}
            response = requests.get(f"{SERVER_URL}/api/industrial-ai/agents", headers=headers)
            print(f"✅ 에이전트 목록: {response.status_code}")
            
            if response.status_code == 200:
                data = response.json().get('data', {})
                print(f"📊 총 에이전트: {data.get('total_agents')}개")
                print(f"📊 활성 에이전트: {data.get('active_agents')}개")
            
            # 4. 주조 분석 테스트
            analysis_data = {
                "problem_type": "defect_analysis",
                "description": "기공 결함 테스트",
                "process_data": {"온도": 780, "압력": 300}
            }
            
            response = requests.post(
                f"{SERVER_URL}/api/industrial-ai/analyze/casting",
                json=analysis_data,
                headers=headers
            )
            print(f"✅ 주조 분석: {response.status_code}")
            
            if response.status_code == 200:
                result = response.json().get('data', {})
                analysis = result.get('analysis', {})
                print(f"🔍 발견된 결함: {len(analysis.get('detected_defects', []))}개")
                print(f"📈 신뢰도: {analysis.get('confidence', 0):.2%}")
    
    except Exception as e:
        print(f"❌ API 테스트 실패: {e}")
    
    print("\n🎉 API 테스트 완료!")

# API 테스트 실행
test_api()

## 🛑 서버 중지

In [ ]:
# 서버 중지 (필요시 실행)
# stop_server()

## 📱 브라우저에서 확인

서버가 정상적으로 실행되면 다음 링크들을 클릭하여 확인하세요:

- **메인 페이지**: http://localhost:8002
- **API 문서**: http://localhost:8002/docs
- **헬스 체크**: http://localhost:8002/api/health
- **상태 정보**: http://localhost:8002/api/status

---

### 🎯 주요 기능

1. **Agno 초경량 에이전트** (3μs 생성, 6.5KB 메모리)
2. **LlamaIndex 워크플로우** 통합
3. **주조 전문 AI 분석** 시스템
4. **실시간 성능 모니터링**
5. **RESTful API** 엔드포인트

### 🔐 테스트 계정
- **사용자명**: admin_001
- **비밀번호**: admin_pass_001